In [3]:
!pip install ucimlrepo


In [4]:
# ===========================================
# Experiment 5: Decision Tree from Scratch (Fixed Version)
# ===========================================

# --- Imports ---

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier as SklearnTree

In [5]:
# ===========================================
# 1. Load and Prepare the Dataset
# ===========================================
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
data = pd.read_csv(url, header=None, names=columns, na_values=' ?', skipinitialspace=True)

# Drop missing values (keep X and y aligned)
data = data.dropna()

# Separate features and target
X = data.drop('income', axis=1)
y = data['income']

# Encode categorical columns
label_encoders = {}
for col in X.select_dtypes(include='object').columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# Encode target
y = LabelEncoder().fit_transform(y)

# Ensure aligned lengths
assert len(X) == len(y), f"X and y length mismatch: {len(X)} vs {len(y)}"

# Split into train-validation-test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

print("Train:", X_train.shape, "Validation:", X_val.shape, "Test:", X_test.shape)

Train: (20838, 14) Validation: (5210, 14) Test: (6513, 14)


In [7]:
# ===========================================
# 2. Decision Tree Implementation from Scratch
# ===========================================
class DecisionTreeScratch:
    def __init__(self, max_depth=None, min_samples_split=2, criterion="gini"):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.tree = None

    def _gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        probs = counts / counts.sum()
        return 1 - np.sum(probs ** 2)

    def _entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probs = counts / counts.sum()
        return -np.sum(probs * np.log2(probs + 1e-9))

    def _impurity(self, y):
        return self._gini(y) if self.criterion == "gini" else self._entropy(y)

    def _best_split(self, X, y):
        best_gain = -1
        best_split = None
        parent_impurity = self._impurity(y)

        for feature in range(X.shape[1]):
            values = np.unique(X[:, feature])
            for val in values:
                left_idx = X[:, feature] <= val
                right_idx = X[:, feature] > val
                if len(y[left_idx]) < self.min_samples_split or len(y[right_idx]) < self.min_samples_split:
                    continue

                left_impurity = self._impurity(y[left_idx])
                right_impurity = self._impurity(y[right_idx])
                n = len(y)
                child_impurity = (len(y[left_idx]) / n) * left_impurity + (len(y[right_idx]) / n) * right_impurity
                info_gain = parent_impurity - child_impurity

                if info_gain > best_gain:
                    best_gain = info_gain
                    best_split = {"feature": feature, "threshold": val, "gain": info_gain}

        return best_split

    def _build_tree(self, X, y, depth=0):
        if len(np.unique(y)) == 1:
            return {"leaf": True, "class": np.unique(y)[0]}
        if self.max_depth is not None and depth >= self.max_depth:
            return {"leaf": True, "class": np.bincount(y).argmax()}

        split = self._best_split(X, y)
        if not split or split["gain"] <= 0:
            return {"leaf": True, "class": np.bincount(y).argmax()}

        left_idx = X[:, split["feature"]] <= split["threshold"]
        right_idx = X[:, split["feature"]] > split["threshold"]

        left_subtree = self._build_tree(X[left_idx], y[left_idx], depth + 1)
        right_subtree = self._build_tree(X[right_idx], y[right_idx], depth + 1)

        return {
            "leaf": False,
            "feature": split["feature"],
            "threshold": split["threshold"],
            "left": left_subtree,
            "right": right_subtree,
        }

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _predict_one(self, x, node):
        if node["leaf"]:
            return node["class"]
        if x[node["feature"]] <= node["threshold"]:
            return self._predict_one(x, node["left"])
        else:
            return self._predict_one(x, node["right"])

    def predict(self, X):
        return np.array([self._predict_one(x, self.tree) for x in X])

In [8]:
# ===========================================
# 3. Train & Evaluate with Different Depths (Pre-Pruning)
# ===========================================
depths = [2, 4, 6, None]
results = {}

for d in depths:
    print(f"\nTraining tree with max_depth = {d}")
    tree = DecisionTreeScratch(max_depth=d, criterion="gini")
    tree.fit(X_train.values, y_train)
    y_pred = tree.predict(X_val.values)
    acc = accuracy_score(y_val, y_pred)
    results[d] = acc
    print("Validation Accuracy:", acc)

best_depth = max(results, key=results.get)
print("\nBest depth based on validation:", best_depth)


Training tree with max_depth = 2
Validation Accuracy: 0.8322456813819578

Training tree with max_depth = 4
Validation Accuracy: 0.8516314779270633

Training tree with max_depth = 6
Validation Accuracy: 0.8571976967370442

Training tree with max_depth = None
Validation Accuracy: 0.8166986564299424

Best depth based on validation: 6


In [9]:
# ===========================================
# 4. Post-Pruning (Reduced Error Pruning)
# ===========================================
print("\nPost-pruning placeholder (conceptual, can be expanded).")


Post-pruning placeholder (conceptual, can be expanded).


In [10]:
# ===========================================
# 5. Final Evaluation
# ===========================================
final_tree = DecisionTreeScratch(max_depth=best_depth, criterion="gini")
final_tree.fit(X_train_full.values, y_train_full)
y_pred_test = final_tree.predict(X_test.values)

print("\n--- Custom Decision Tree Results ---")
print("Accuracy:", accuracy_score(y_test, y_pred_test))
print("Precision:", precision_score(y_test, y_pred_test))
print("Recall:", recall_score(y_test, y_pred_test))
print("F1 Score:", f1_score(y_test, y_pred_test))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))

# ===========================================
# 6. Compare with sklearn DecisionTreeClassifier
# ===========================================
clf = SklearnTree(max_depth=best_depth, criterion="gini")
clf.fit(X_train_full, y_train_full)
y_pred_sklearn = clf.predict(X_test)

print("\n--- Sklearn Decision Tree Results ---")
print("Accuracy:", accuracy_score(y_test, y_pred_sklearn))
print("Precision:", precision_score(y_test, y_pred_sklearn))
print("Recall:", recall_score(y_test, y_pred_sklearn))
print("F1 Score:", f1_score(y_test, y_pred_sklearn))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_sklearn))


--- Custom Decision Tree Results ---
Accuracy: 0.8547520343927529
Precision: 0.8013500482160077
Recall: 0.5289624443029918
F1 Score: 0.6372699386503068
Confusion Matrix:
 [[4736  206]
 [ 740  831]]
              precision    recall  f1-score   support

           0       0.86      0.96      0.91      4942
           1       0.80      0.53      0.64      1571

    accuracy                           0.85      6513
   macro avg       0.83      0.74      0.77      6513
weighted avg       0.85      0.85      0.84      6513


--- Sklearn Decision Tree Results ---
Accuracy: 0.8549055734684478
Precision: 0.8021235521235521
Recall: 0.5289624443029918
F1 Score: 0.6375143843498274
Confusion Matrix:
 [[4737  205]
 [ 740  831]]
